In [1]:
import sqlite3
import json
from datetime import datetime

In [2]:
timeframe = '2015-01'
sql_transaction = []

connection = sqlite3.connect('{}.db'.format(timeframe))
c = connection.cursor()

In [3]:
def create_table():
    c.execute("""CREATE TABLE IF NOT EXISTS parent_reply(parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, 
              parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT)""")

In [4]:
def format_data(data):
    data = data.replace('\n',' newlinechar ').replace('\r',' newlinechar ').replace('"',"'")
    return data

In [5]:
def find_parent(pid):
    try:
        sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False

In [6]:
def find_existing_score(pid):
    try:
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False

In [7]:
def acceptable(data):
    if len(data.split(' ')) > 50 or len(data) < 1:
        return False
    elif len(data) > 1000:
        return False
    elif data == '[deleted]':
        return False
    elif data == '[removed]':
        return False
    else:
        return True

In [8]:
def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
        connection.commit()
        sql_transaction = []

In [9]:
def sql_insert_replace_comment(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """UPDATE parent_reply SET parent_id = ?, comment_id = ?, parent = ?, comment = ?, subreddit = ?, unix = ?, score = ? WHERE parent_id =?;""".format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_bldr(sql)
    except Exception as e:
        print('replace comment',str(e))

In [10]:
def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('insertion in parent comment',str(e))

In [11]:
def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}",{},{});""".format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('insertion with no comment',str(e))

In [12]:
if __name__ == '__main__':
    create_table()
    row_counter = 0
    paired_rows = 0
    with open('RC_{}'.format(timeframe), buffering=1000) as f:
        for row in f:
            row_counter += 1
            row = json.loads(row)
            parent_id = row['parent_id']
            body = format_data(row['body'])
            created_utc = row['created_utc']
            score = row['score']
            comment_id = row['name']
            subreddit = row['subreddit']
            parent_data = find_parent(parent_id)
            
            if score >= 2:
                if acceptable(body):
                    existing_comment_score = find_existing_score(parent_id) 
                    if existing_comment_score:
                        if score > existing_comment_score:
                            sql_insert_replace_comment(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                    else:
                        if parent_data:
                            sql_insert_has_parent(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                            paired_rows += 1
                        else:
                            sql_insert_no_parent(comment_id,parent_id,body,subreddit,created_utc,score)
            if row_counter % 100000 == 0:
                print('Total Rows Read: {}, Paired Rows: {}, Time: {}'.format(row_counter, paired_rows, str(datetime.now())))

Total Rows Read: 100000, Paired Rows: 3718, Time: 2018-08-10 22:46:36.994342
Total Rows Read: 200000, Paired Rows: 8432, Time: 2018-08-10 22:47:11.587596
Total Rows Read: 300000, Paired Rows: 13772, Time: 2018-08-10 22:47:54.730088
Total Rows Read: 400000, Paired Rows: 19179, Time: 2018-08-10 22:48:37.937785
Total Rows Read: 500000, Paired Rows: 24365, Time: 2018-08-10 22:49:20.873713
Total Rows Read: 600000, Paired Rows: 28878, Time: 2018-08-10 22:49:49.822862
Total Rows Read: 700000, Paired Rows: 33611, Time: 2018-08-10 22:50:20.142222
Total Rows Read: 800000, Paired Rows: 39023, Time: 2018-08-10 22:50:52.397254
Total Rows Read: 900000, Paired Rows: 44662, Time: 2018-08-10 22:51:25.349501
Total Rows Read: 1000000, Paired Rows: 50247, Time: 2018-08-10 22:51:58.392212
Total Rows Read: 1100000, Paired Rows: 55890, Time: 2018-08-10 22:52:32.279965
Total Rows Read: 1200000, Paired Rows: 61380, Time: 2018-08-10 22:53:04.746766
Total Rows Read: 1300000, Paired Rows: 67071, Time: 2018-08-10 

Total Rows Read: 10400000, Paired Rows: 581946, Time: 2018-08-11 00:04:55.269083
Total Rows Read: 10500000, Paired Rows: 587347, Time: 2018-08-11 00:06:29.260650
Total Rows Read: 10600000, Paired Rows: 592208, Time: 2018-08-11 00:07:56.437530
Total Rows Read: 10700000, Paired Rows: 597088, Time: 2018-08-11 00:09:13.489645
Total Rows Read: 10800000, Paired Rows: 602615, Time: 2018-08-11 00:10:23.468703
Total Rows Read: 10900000, Paired Rows: 608336, Time: 2018-08-11 00:11:35.447251
Total Rows Read: 11000000, Paired Rows: 614359, Time: 2018-08-11 00:12:46.997756
Total Rows Read: 11100000, Paired Rows: 620388, Time: 2018-08-11 00:13:57.274123
Total Rows Read: 11200000, Paired Rows: 626342, Time: 2018-08-11 00:15:04.091114
Total Rows Read: 11300000, Paired Rows: 632509, Time: 2018-08-11 00:16:06.480129
Total Rows Read: 11400000, Paired Rows: 638744, Time: 2018-08-11 00:17:10.092372
Total Rows Read: 11500000, Paired Rows: 644898, Time: 2018-08-11 00:18:18.617651
Total Rows Read: 11600000, P

KeyboardInterrupt: 

In [13]:
# Create database as much you want but should be more than 100,000pairs
# it took me more than 2 hours for 802171 pairs